In [15]:
import re
from typing import Tuple
import numpy as np
from SolvingAdvent.HelferWichtel import HelferWichtel as Wichtel

TagFile = '09'


Helfer = Wichtel()
PuzzleInput = ''.join(Helfer.getPuzzleInput(TagFile))
PuzzleInput = [line.replace(' ','') for line in PuzzleInput.split('\n')]
# print([line.replace(' ','') for line in PuzzleInput])


In [28]:
class World:
    def __init__(self):
        xvalues = np.arange(-5,5)
        yvalues = np.arange(5,-5,-1)
        self.xx, self.yy = np.meshgrid(xvalues,yvalues)
        self.headPosition = np.array([0,0])
        self.tailPosition = np.array([0,0])
        self.tailHistory = np.array([[0],[0]])
        self.directionDict = {'U':[0,1],'R':[1,0],'D':[0,-1],'L':[-1,0]}
        
    def getWorld(self):
        return self.xx, self.yy

    def getHeadKnotPosition(self):
        return self.headPosition

    def getTailKnotPosition(self):
        return self.tailPosition

    def getTailHistory(self):
        return self.tailHistory
    
    def extendMapNorth(self):
        self.xx = np.vstack([self.xx[0],self.xx])
        self.yy = np.vstack([self.yy[0]+1,self.yy])
        
    def extendMapEast(self):
        self.xx = np.hstack([self.xx, np.transpose([self.xx[:,-1]+1])])
        self.yy = np.hstack([self.yy, np.transpose([self.yy[:,-1]])])
        
    def extendMapSouth(self):
        self.xx = np.vstack([self.xx, self.xx[-1]])
        self.yy = np.vstack([self.yy, self.yy[-1]-1])
        
    def extendMapWest(self):
        self.xx = np.hstack([np.transpose([self.xx[:,0]-1]), self.xx])
        self.yy = np.hstack([np.transpose([self.yy[:,0]]), self.yy])
        
    def getArrayIndexOfCoordinate(self, x:int, y:int) -> Tuple[int, int]:
        xcoordinate = int(np.where(self.xx == x)[1][0])
        ycoordinate = np.where(self.yy == y)[0][0]
        return xcoordinate, ycoordinate

    def moveHeadKnot(self,x,y): 
        '''Move head knot in given direction for one field'''
        if not ((abs(x)<=1)&(abs(y)<=1)):
            raise Exception('Nur ein Feld Bewegung erlaubt!')
        newHeadPosition = self.headPosition + np.array([x,y])
        self.headPosition = newHeadPosition
        self.moveTailKnot()

    def moveTailKnot(self):
        moveX = 0
        moveY = 0
        headPosition = self.headPosition
        tailPosition = self.tailPosition
        deltaX = headPosition[0] - tailPosition[0]
        deltaY = headPosition[1] - tailPosition[1]
        # Diagonal
        if (deltaX!=0)&(deltaY!=0):
            # not touching
            if(abs(deltaX)>1)|(abs(deltaY)>1):
                moveX = np.sign(deltaX)
                moveY = np.sign(deltaY)
        # Not Diagonal
        else:
            if deltaX > 1:
                moveX = 1
            if deltaX < -1:
                moveX = -1
            if deltaY > 1:
                moveY = 1
            if deltaY < -1:
                moveY = -1
        self.tailPosition = tailPosition+ np.array([moveX,moveY])
        self.tailHistory = np.hstack([self.tailHistory,np.transpose([self.tailPosition])])

    def getNumberOfTailPositions(self):
        history = [tuple(column) for column in self.getTailHistory().T]
        return len(np.unique(history,axis=0))

    def executeLine(self,line):
        direction = re.search('\D',line).group()
        distance = int(re.search('\d+',line).group())
        for step in range(0,distance):
            moveX = self.directionDict[direction][0]
            moveY = self.directionDict[direction][1]
            self.moveHeadKnot(moveX,moveY)
        return distance
        





In [33]:
# Aufgabe A
newWorld = World()
Summe = 0
for line in PuzzleInput:
    Summe += newWorld.executeLine(line)
print('Antwort von Tag 9 A ist: %s' %(newWorld.getNumberOfTailPositions()))
print('Anzahl Schritte von Head in der Liste: %s' %Summe)




Antwort von Tag 9 A ist: 6406
Anzahl Schritte von Head in der Liste: 11532


In [118]:
class Seil():
    def __init__(self,numberOfKnots = 2):
        self.numberOfKnots = numberOfKnots
        self.knotPositions = [np.array([0,0]) for n in range(0,numberOfKnots)]
        self.tailHistory = np.array([[0],[0]])
        self.directionDict = {'U':[0,1],'R':[1,0],'D':[0,-1],'L':[-1,0]}

    def getHeadPosition(self):
        return self.knotPositions[0]

    def getKnotPosition(self,knotNumber):
        return self.knotPositions[knotNumber]

    def getTailHistory(self):
        return self.tailHistory

    def moveHeadKnot(self,x,y): 
        '''Move head knot in given direction for one field'''
        if not ((abs(x)<=1)&(abs(y)<=1)):
            raise Exception('Nur ein Feld Bewegung erlaubt!')
        newHeadPosition = self.getHeadPosition() + np.array([x,y])
        self.knotPositions[0] = newHeadPosition
        for n in range(1,self.numberOfKnots):
            self.moveTrailingKnot(n)

    def moveTrailingKnot(self,knotNumber):
        moveX = 0
        moveY = 0
        leadingKnotPosition = self.knotPositions[knotNumber-1]
        knotPosition = self.knotPositions[knotNumber]
        deltaX = leadingKnotPosition[0] - knotPosition[0]
        deltaY = leadingKnotPosition[1] - knotPosition[1]
        # Diagonal
        if (deltaX!=0)&(deltaY!=0):
            # not touching
            if(abs(deltaX)>1)|(abs(deltaY)>1):
                moveX = np.sign(deltaX)
                moveY = np.sign(deltaY)
        # Not Diagonal
        else:
            if deltaX > 1:
                moveX = 1
            if deltaX < -1:
                moveX = -1
            if deltaY > 1:
                moveY = 1
            if deltaY < -1:
                moveY = -1
        knotPosition = knotPosition + np.array([moveX,moveY])
        self.knotPositions[knotNumber] = knotPosition
        if knotNumber == self.numberOfKnots-1:
            self.tailHistory = np.hstack([self.tailHistory,np.transpose([knotPosition])])

    def getNumberOfTailPositions(self):
        history = [tuple(column) for column in self.getTailHistory().T]
        return len(np.unique(history,axis=0))

    def executeLine(self,line):
        direction = re.search('\D',line).group()
        distance = int(re.search('\d+',line).group())
        for step in range(0,distance):
            moveX = self.directionDict[direction][0]
            moveY = self.directionDict[direction][1]
            self.moveHeadKnot(moveX,moveY)
        return distance

In [120]:
newSeil = Seil(10)
Summe = 0
for line in PuzzleInput:
    Summe += newSeil.executeLine(line)
print('Antwort von Tag 9 B ist: %s' %(newSeil.getNumberOfTailPositions()))
print('Anzahl Schritte von Head in der Liste: %s' %Summe)

Antwort von Tag 9 B ist: 2643
Anzahl Schritte von Head in der Liste: 11532
